In [1]:
# For google collab:
# 1.unzip samples/samples on your local machine. And zip it to samples.zip
# 2.load samples.zip to collab
# 3.run:


# !unzip -q /content/samples.zip

### Capthca solver model via convolutional neural network

It is time to reveal the truth: I am Russian.... 
##### Методология построения модели: 
1. Так как можель с изображениями, входное изображение подается на вход трем сверточным полносвязным слоям.
2. Перед подачей изображения сверточному слою считаются средние и дисперсия по тренировчному набору, затем данные нормируются.
3. После каждого свертоночного слоя идут пулинг и батчнорм.
4. После последнего сверточного слоя матрица раскрывается в один вектор. Этот вектор поступает на входя пяти параллельным независимым полносвязнанным нейронным сетям с двумя слоями. Каждый слой отвечает за распознавание своей (по счету) буквы.
5. Сеть выдает 5 векторов размера 36 (строчные буквы + цифры), каждый выходной вектор сравнивается с меткой класса = букве на соответствующем месте капчи. Критерий -- кросс-энтроопия. Будем минимизировать сумму пяти значений loss функций.

##### Методология подбора параметров:
1. Построить как можно более сложную сеть, способную обучиться и переобучиться на тренировочной выборке. 
2. Посредством аугментации входных изображений и дропаута в слоях добиваться уменьшения  переобучения на тестовых данных.

Методы аугментации.
Так как изображение небольшое, искажения дролжны быть небольшие. Из афинных преобразований сразу исклчается сдвиг, так как модель должна примерно понимать, на каком месте стоит какая буква, по то же причине исключается растяжение. Остается только поворот. 
Более сложные аугментации, такие как ElasticTransform сильно замедляют загрузку данных и как следствие обучение модели 

##### Качество модели: 
Лучший результат, которого удалось добиться на тестовой выборке:  <br>
`Test characters error rate = 11.8 %`

##### Анализ ошибок:
Не вижу смысла проводить анализ ошибок, так как и в чате говорилось, и преподаватели говорили, что можно достичь ошибки меньше 10 и даже меньше 5 процентов. После всех моих усилий мне это не удалось. Большая просьба к проверяющим, подсказать, как улучшить архитектуру моей сети.


This notebook doesn't contain opportunity of flexible configuration. It might be done by using python-code from `/python` folder and changing `/config/config.yaml`

Libraries:

In [2]:
import os 
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import string
import torchvision.transforms as transforms
import torchsummary
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Data processing functions

In [3]:
def get_train_test_names(root_dir, test_size=0.2):
    """
    Create train and test dataset with filenames and labels
    """
    picture_names = [[picture_name, picture_name[: -4]] for picture_name
                     in os.listdir(root_dir)]

    names_dataframe = pd.DataFrame(picture_names, columns=('filename', 'label'))
    names_train, names_test = train_test_split(names_dataframe, 
                                   test_size=test_size, 
                                   random_state=42)
    return names_train.reset_index(drop=True), names_test.reset_index(drop=True)


def get_mean_std(dataset_class, names_train, path, transform):
    """
    Get mean and std over all training dataset
    """
    dataset = dataset_class(names_train, PATH, transform)
    train_tensors = [dataset[i][0] for i in range(len(names_train))]
    train_tensors_stack = torch.stack(train_tensors)
    return train_tensors_stack.mean(), train_tensors_stack.std()


Model evaluation function

In [4]:
def tensor_to_word(dateched_tensor):
    dataset = CaptchaDataset(pd.DataFrame(), '')
    symbols_array = tuple(map(lambda x: dataset.id_to_symbol[int(x)], 
              tuple(dateched_tensor)))
    return ''.join(map(str, symbols_array))



def character_error_rate(net, loader, verbose=False):
    """
    Model quality evaluation
    enable verbose to see word pairs: errors in recognition 
    """
    net.train(False)
    with torch.no_grad():
        errors = 0
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs[:,0,:], 1)
            predicted, labels = predicted.detach().cpu().numpy(), \
                    labels.detach().cpu().numpy()[0]
            
            errors += np.count_nonzero(predicted - labels)
            if verbose:
                print("error:", tensor_to_word(predicted),
                 tensor_to_word(labels))
                
    return errors / (LETTERS_AMOUNT * len(loader))

Model configuration and training routine  

In [5]:
def train_model(net, criterion, optimizer, trainloader, num_epochs=10):
    net.train(True)
    for epoch in range(num_epochs):
        print(f"{epoch=}")
        running_loss = 0.0
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = net(inputs).to(device)
            loss = 0
            for letter_num in range(LETTERS_AMOUNT):
                loss += criterion(outputs[:, letter_num], labels[letter_num])
            loss.backward()
            optimizer.step()
        print(f'{loss=}')
    print('Finished Training')
    return net



class CaptchaDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None, verbose=False):
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.verbose = verbose
        self.symbols = list(map(str, range(10))) + list(string.ascii_lowercase) 
        self.symbol_to_id = {key: value for value, key in enumerate(self.symbols)}
        self.id_to_symbol = {value: key for value, key in enumerate(self.symbols)}

    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
            
        
        img_name = os.path.join(self.root_dir, self.dataframe['filename'][index])
        image = Image.open(img_name).convert('L')
        
        label = torch.tensor(np.array(
            [self.symbol_to_id[idx] for idx in self.dataframe['label'][index]]))   
        
        if self.transform:
            img_tensor = self.transform(image)
        
        return img_tensor, label
    

class CaptchaSolverNet(nn.Module):
    def __init__(self):
        super(CaptchaSolverNet, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 128, kernel_size=(3, 6), padding=(1, 1))
        self.conv2 = nn.Conv2d(128, 64, kernel_size=(3, 6), padding=(1, 1))
        self.conv3 = nn.Conv2d(64, 32, kernel_size=(3, 6), padding=(1, 1))

        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.bn1 = nn.BatchNorm2d(128)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm1d(512)

        # Full connected layers for each letter
        self.partials1 = torch.nn.ModuleList(tuple((
            nn.Linear(in_features=4224, out_features=512) 
            for _ in range(LETTERS_AMOUNT))))


        self.partials2 = torch.nn.ModuleList(tuple((
            nn.Linear(in_features=512, out_features=36) 
            for _ in range(LETTERS_AMOUNT))))



        self.dropout = nn.Dropout(0.1)


    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.bn1(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.bn2(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.bn3(x)


        x = torch.flatten(x, 1)
        outputs = []
        for letter_num in range(LETTERS_AMOUNT):
            letter = self.partials1[letter_num](x)
            letter = F.relu(letter)
            letter = self.bn4(letter)
            letter = self.dropout(letter)

            letter = self.partials2[letter_num](letter)
            letter = F.relu(letter)

            outputs.append(letter)
        result = torch.stack(outputs)
        return result


### __main__

Configure path to data

In [11]:
folder_path = os.getcwd() 
PATH = os.path.abspath(os.path.join(folder_path, os.pardir, 'samples', 'samples'))



# uncomment if working in collab: 
# PATH = '/content/samples'

Configure device

In [12]:
if torch.cuda.is_available(): 
     dev = "cuda:0" 
else: 
     dev = "cpu" 
device = torch.device(dev) 
print(f'{device=}')

device=device(type='cuda', index=0)


Since we solve 5-letters captcha set

In [13]:
LETTERS_AMOUNT = 5

Read data from file, calcute statistics, transform, normalize

In [14]:
transform_to_tensor = transforms.Compose([
    transforms.ToTensor()])


names_train, names_test = get_train_test_names(PATH)
sample_mean, sample_std = get_mean_std(CaptchaDataset, names_train, 
                                       PATH, transform_to_tensor)

transform_to_tensor_and_norm_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(sample_mean, sample_std),
    transforms.RandomAffine(degrees=(5))])

transform_to_tensor_and_norm_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(sample_mean, sample_std)])

Conffgure datasets and dataloaders

In [15]:
trainset = CaptchaDataset(names_train, PATH, transform_to_tensor_and_norm_train)
testset = CaptchaDataset(names_test, PATH, transform_to_tensor_and_norm_test,
                        verbose=True)

batch_size = 24
num_workers = 2

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=num_workers)

Training model:

In [16]:
net = CaptchaSolverNet()
net = net.to(device)

torchsummary.summary(net, (1, 50, 200))

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-4, momentum=0.9)

net = train_model(net, criterion, optimizer, trainloader, num_epochs=150)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 128, 50, 197]           2,432
         MaxPool2d-2          [-1, 128, 25, 98]               0
       BatchNorm2d-3          [-1, 128, 25, 98]             256
            Conv2d-4           [-1, 64, 25, 95]         147,520
         MaxPool2d-5           [-1, 64, 12, 47]               0
       BatchNorm2d-6           [-1, 64, 12, 47]             128
            Conv2d-7           [-1, 32, 12, 44]          36,896
         MaxPool2d-8            [-1, 32, 6, 22]               0
       BatchNorm2d-9            [-1, 32, 6, 22]              64
           Linear-10                  [-1, 512]       2,163,200
      BatchNorm1d-11                  [-1, 512]           1,024
          Dropout-12                  [-1, 512]               0
           Linear-13                   [-1, 36]          18,468
           Linear-14                  [

Evaluate model quality

In [17]:
batch_size = 1
num_workers = 1

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                         shuffle=False, num_workers=num_workers)


print(f'Trian characters error rate = {character_error_rate(net, trainloader)}')
print(f'Test characters error rate = {character_error_rate(net, testloader)}')




# """
# Uncomment below to see missclassification pairs
# """
# print(f'Trian characters error rate = {character_error_rate(net, 
# trainloader, verbose=True)}')
# print(f'Test characters error rate = {character_error_rate(net, 
# testloader, verbose=True)}')



Trian characters error rate = 0.021261682242990656
Test characters error rate = 0.13271028037383178


In [20]:
# on local machine:
torch.save(net.state_dict(), os.path.abspath(os.path.join(folder_path, os.pardir, 
                                                          'result_model','result')))



# On collab
# torch.save(net.state_dict(), 'result')